In [3]:
import pandas as pd
import glob
import os
import datetime
import pprint
pp = pprint.PrettyPrinter(indent=4)

import numpy as np

In [ ]:
sensorFolder = 'spl-deploy-rc3flr'

In [5]:
#selecting all the csv files
files = os.path.join(sensorFolder,"2022*.csv")

# list of merged files returned
files = glob.glob(files)

#sorting files by date
files.sort()
# pp.pprint(files)

#checking a single file
print(files[0])

2022-05-18.csv


In [7]:
print("Resultant CSV after joining all CSV files at a particular location...");
# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), ignore_index=True)
print(df)

Resultant CSV after joining all CSV files at a particular location...
         Unnamed: 0   laeq   lceq   lzeq          time
0                 0  64.35  72.85  73.29  1.652933e+09
1                 1  64.44  73.43  73.85  1.652933e+09
2                 2  64.08  72.59  73.08  1.652933e+09
3                 3  64.06  71.71  72.36  1.652933e+09
4                 4  63.41  71.62  72.22  1.652933e+09
...             ...    ...    ...    ...           ...
1930958         115  61.96  72.26  73.65  1.655093e+09
1930959         116  61.29  73.09  74.81  1.655093e+09
1930960         117  58.99  73.20  74.77  1.655093e+09
1930961         118  58.27  74.80  76.56  1.655093e+09
1930962         119  58.17  73.69  75.72  1.655093e+09

[1930963 rows x 5 columns]


In [8]:
#converting epoch time to datetime
df['time'] = df.time.apply(datetime.datetime.fromtimestamp)
# df['time'] = pd.to_datetime(df['time'], unit = 's')
df.rename(columns={'time': 'datetime'}, inplace=True)


In [9]:
#adding additional columns for datetime 

#extracting exact date
df['date'] = df['datetime'].dt.date

#extracting day of week
df['dayofweek'] = df['datetime'].dt.dayofweek
# df['dayofweek_name'] = df['datetime'].dt.day_name

#extracting extra data
# df['hour'] = df['datetime'].dt.hour
# df['min'] = df['datetime'].dt.minute
# df['sec'] = df['datetime'].dt.second

In [10]:
#checking file
print(df)

         Unnamed: 0   laeq   lceq   lzeq                datetime        date  \
0                 0  64.35  72.85  73.29 2022-05-18 23:59:16.155  2022-05-18   
1                 1  64.44  73.43  73.85 2022-05-18 23:59:16.898  2022-05-18   
2                 2  64.08  72.59  73.08 2022-05-18 23:59:18.392  2022-05-18   
3                 3  64.06  71.71  72.36 2022-05-18 23:59:19.126  2022-05-18   
4                 4  63.41  71.62  72.22 2022-05-18 23:59:19.871  2022-05-18   
...             ...    ...    ...    ...                     ...         ...   
1930958         115  61.96  72.26  73.65 2022-06-13 00:01:54.668  2022-06-13   
1930959         116  61.29  73.09  74.81 2022-06-13 00:01:56.150  2022-06-13   
1930960         117  58.99  73.20  74.77 2022-06-13 00:01:56.893  2022-06-13   
1930961         118  58.27  74.80  76.56 2022-06-13 00:01:57.638  2022-06-13   
1930962         119  58.17  73.69  75.72 2022-06-13 00:01:59.125  2022-06-13   

         dayofweek  
0                2

In [11]:
#saving df to csv
#df.to_csv('masterdata.csv') uncomment to save file locally

In [108]:
#extracting day of week & time of day labels
timeofdayDict = {0: 'Early Morning (12am-6am)', 
                 6: 'Morning (6am-12pm)', 
                 12: 'Afternoon (12pm-6pm)', 
                 18: 'Night (6pm-12am)'}

TDlabels = list(timeofdayDict.values())
# print(timeofdayDict)

daysofweekDict = {0:'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 
                  4: 'Friday', 5: 'Saturday',6: 'Sunday'}

DWlabels = list(daysofweekDict.values())


# timeofdayDict[12]
# daysofweekDict[4]

### Objective Dates

In [109]:
"""
FINDING MIN & MAX AVERAGE LAEQ LEVEL DAY OF ALL DATES
"""
DTaggs = df.groupby(['date']).agg({'laeq': ['mean', 'max']})
DTaggs.columns = ['laeq_mean', 'laeq_max']
DTaggs = DTaggs.reset_index()

mean_min = DTaggs.laeq_mean.min()
mean_minDay = DTaggs.loc[DTaggs['laeq_mean'].idxmin(), 'date']


print("Minimum LAeq", mean_min, "and its corresponding date:", mean_minDay)

mean_max = DTaggs.laeq_mean.max()
mean_maxDay = DTaggs.loc[DTaggs['laeq_mean'].idxmax(), 'date']

# print(mean_max, mean_maxDay)
print("Maximum LAeq", mean_max, "and its corresponding date:", mean_maxDay)

# print("Maximum value of column ", col, " and its corresponding row values:\n", max_x)



Minimum LAeq 61.51616386194722 and its corresponding date: 2022-06-05
Maximum LAeq 66.44829100529083 and its corresponding date: 2022-05-20


In [148]:
# print(df)

"""
FINDING MAX LAEQ LEVEL OF ALL DATES
"""
absMax = df.laeq.max()
absMax
absMaxInfo = df.loc[df['laeq'].idxmax()]
time_absMax = absMaxInfo.name.time()

# print(absMaxInfo)

print('Loudest Day:',absMaxInfo['date'], 
      '\nDay of Week:', daysofweekDict[absMaxInfo['dayofweek']],
     '\nTime:', time_absMax)

Loudest Day: 2022-06-13 
Day of Week: Monday 
Time: 11:51:16.049000


### Day of Week

In [114]:
"""
FINDING MIN & MAX AVG LAEQ LEVEL DAY OF WEEK
"""
DWaggs = df.groupby(['dayofweek']).agg({'laeq': ['mean', 'max']})
DWaggs.columns = ['DWlaeq_mean', 'DWlaeq_max']
DWaggs = DWaggs.reset_index()

DWmean_min = DWaggs.DWlaeq_mean.min()
DWmean_minDay = DWaggs.loc[DWaggs['DWlaeq_mean'].idxmin(), 'dayofweek']

print("Minimum LAeq", DWmean_min, "and its corresponding day of week:", daysofweekDict[DWmean_minDay])

DWmean_max = DWaggs.DWlaeq_mean.max()
DWmean_maxDay = DWaggs.loc[DWaggs_max['DWlaeq_mean'].idxmax(), 'dayofweek']

# print(DWmean_max, DWmean_maxDay)

print("Maximum LAeq", DWmean_max, "and its corresponding day of week:", daysofweekDict[DWmean_maxDay])



Minimum LAeq 61.750916724371315 and its corresponding day of week: Sunday
Maximum LAeq 63.6004801983995 and its corresponding day of week: Thursday


### Time of Day

In [117]:
"""
FINDING MIN & MAX AVG LAEQ LEVEL DAY OF WEEK
"""
#df.set_index('datetime',inplace=True) #only need to do this one
TDdf = df.resample('6H').mean();
# print(x)
# print(x.index.hour)
TDdf['hour'] = TDdf.index.hour;
TDdf_mean = TDdf.groupby(['dayofweek', 'hour']).agg({'laeq': ['mean']})

TDdf_mean.columns = ['TDlaeq_mean']
TDdf_mean = TDdf_mean.reset_index()

In [116]:
hours_ref=TDdf['hour']
hourlist1=hours_ref.unique().tolist()
hrlist = sorted(hourlist1)


In [105]:
# print(TDdf_mean)

TDmean_min = TDdf_mean.TDlaeq_mean.min()
TDmean_minHR = TDdf_mean.loc[TDdf_mean['TDlaeq_mean'].idxmin(), 'hour']
TDmean_minDW = TDdf_mean.loc[TDdf_mean['TDlaeq_mean'].idxmin(), 'dayofweek']

print('Quietest Time and Day')
print(TDmean_min, timeofdayDict[TDmean_minHR],daysofweekDict[TDmean_minDW])

TDmean_max = TDdf_mean.TDlaeq_mean.max()
TDmean_maxHR = TDdf_mean.loc[TDdf_mean['TDlaeq_mean'].idxmax(), 'hour']
TDmean_maxDW = TDdf_mean.loc[TDdf_mean['TDlaeq_mean'].idxmax(), 'dayofweek']

print('Loudest Time and Day')
print(TDmean_max, timeofdayDict[TDmean_maxHR], daysofweekDict[TDmean_maxDW])

Quietest Time and Day
59.70269649148263 Early Morning (12am-6am) Sunday
Loudest Time and Day
65.71119381341273 Afternoon (12pm-6pm) Friday
